In [1]:
!pip install ultralytics opencv-python pyyaml

In [ ]:
import cv2
import numpy as np
import yaml

# Load class names from a YAML file (YOLO format)
def load_classes(path):
    try:
        with open(path, 'r') as f:
            return yaml.safe_load(f).get('names', [])
    except:
        return ['person', 'car', 'chair', 'bottle', 'bird', 'sofa', 'cycle', 'hosre', 'bus']

def detect_objects(net, frame, size=640, conf_th=0.4, nms_th=0.45):
    h, w = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (size, size), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward()[0]

    boxes, confidences, class_ids = [], [], []

    # Iterate over all predictions
    for det in outputs:
        score = det[4] * np.max(det[5:])
        if score > conf_th:
            cx, cy, bw, bh = det[:4]
            x = int((cx - bw/2) * w / size)
            y = int((cy - bh / 2) * h / size)
            boxes.append(float(score))
            confidences.append(float(score))
            class_ids.append(np.argmax(det[5:]))
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, conf_th, nms_th)
    return boxes, confidences, class_ids, idxs

# Draw bounding boxes and class labels on the frame
def draw_detections(frame, boxes, confidencdes, class_ids, indices, names):
    for i in indices.flatten() if len(indices) else []:
        x,y,w,h = boxes[i]
        label = f"{names[class_ids[i]]} {confidencdes[i]:.2f}" if class_ids[i] < len(names) else  f"class_{class_ids[i]}"
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y -5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def main():
    model = r'/home/secret/Documents/pythonVirtualEnv/Model/weights/best.onnx'
    yaml = r'/home/secret/Documents/pythonVirtualEnv/data.yaml'

    class_names = load_classes(yaml)

    net = cv2.dnn.readNetFromONNX(model)
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        boxes, confidences, class_ids, indices = detect_objects(net, frame)

        draw_detections(frame, boxes, confidences, class_ids, indices, class_names)

        cv2.imshow("Dectetion:", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()




In [1]:
import cv2
import numpy as np
import onnxruntime as ort

# Load class names (USE TXT, NOT YAML)
def load_classes(path):
    with open(path) as f:
        return f.read().splitlines()

# Preprocess
def preprocess(frame, size=640):
    img = cv2.resize(frame, (size, size))
    img = img.astype(np.float32) / 255.0
    img = img.transpose(2, 0, 1)  # HWC → CHW
    return np.expand_dims(img, axis=0)

# Postprocess
def postprocess(outputs, frame_shape, conf_th=0.4, nms_th=0.45):
    h, w = frame_shape
    preds = outputs[0][0]

    boxes, scores, class_ids = [], [], []

    for det in preds:
        obj_conf = det[4]
        class_scores = det[5:]
        class_id = np.argmax(class_scores)
        score = obj_conf * class_scores[class_id]

        if score > conf_th:
            cx, cy, bw, bh = det[:4]
            x = int((cx - bw / 2) * w / 640)
            y = int((cy - bh / 2) * h / 640)
            bw = int(bw * w / 640)
            bh = int(bh * h / 640)

            boxes.append([x, y, bw, bh])
            scores.append(float(score))
            class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, scores, conf_th, nms_th)
    return boxes, scores, class_ids, indices

def main():
    model_path = "Model/weights/best.onnx"
    classes_path = "classes.txt"

    class_names = load_classes(classes_path)

    # ONNX Runtime session
    session = ort.InferenceSession(
        model_path, providers=["CPUExecutionProvider"]
    )
    input_name = session.get_inputs()[0].name

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        blob = preprocess(frame)
        outputs = session.run(None, {input_name: blob})

        boxes, scores, class_ids, indices = postprocess(
            outputs, frame.shape[:2]
        )

        if len(indices) > 0:
            for i in indices.flatten():
                x, y, w, h = boxes[i]
                label = f"{class_names[class_ids[i]]}: {scores[i]:.2f}"
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow("YOLOv5 ONNX Runtime", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [8]:
import yaml

with open("data.yaml", "r") as f:
    data = yaml.safe_load(f)

print(data["names"])


['person', 'car', 'chair', 'bottle', 'bird', 'sofa', 'cycle', 'horse', 'bus']


In [ ]:
!pip install onnxruntime


In [13]:
import onnxruntime as ort

session = ort.InferenceSession(
    "Model/weights/best.onnx",
    providers=["CPUExecutionProvider"]
)
